In [1]:
import PyPDF2
import nltk
from nltk.tokenize import word_tokenize
import re
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

In [2]:
# Funcion para convertir el contenido de un pdf a txt
def getPDFFileContentToTXT(pdfFile):
    myPDFFile = PyPDF2.PdfFileReader(pdfFile)
    
#Creamos un archivo txt con la informacion del pdf 
    with open('pdfContenido.txt', 'w') as pdf_output:
        for page in range (myPDFFile.getNumPages()):
            data = myPDFFile.getPage(page).extractText()
            pdf_output.write(data)

# Leemos la infromacion que acabamos de escribir e quitamos los espacios de lineas
    with open('pdfContenido.txt', 'r') as myPDFContent:
        return myPDFContent.read().replace('\n',' ')

## Programa para analizar y calificar CV´s por Andres Saldaña

### Requerimientos de Vacantes

In [3]:
df = pd.read_csv('Requisitos_Vacantes.csv')

In [4]:
df = df.drop("Otros",axis = 1)

In [5]:
df = df.fillna(" ") #rellenamos las casillas sin nada con un espacio

In [6]:
df

,Igualador,Facturacion
0,Igualación de colores,Facturación de clientes
1,Captura de bitácoras,Cobro de ventas mostrador
2,Trabajo en equipo,Elaboración de corte diario
3,Pintura automotriz,Elaboración de Pedidos
4,,Encargado de facturación


In [7]:
#Que vacante quieres cargar?

vacante = "Igualador"
df[vacante]

0    Igualación de colores
1     Captura de bitácoras
2        Trabajo en equipo
3       Pintura automotriz
4                         
Name: Igualador, dtype: object

### Hacer una lista con los requerimientos de la vacante 

In [8]:
requerimientos = ""

for i in df[vacante]:
    requerimientos = requerimientos +i + " "
    
print(requerimientos)

Igualación de colores Captura de bitácoras Trabajo en equipo Pintura automotriz   


### Convertir CV a TXT

In [9]:
pdfFileContent = getPDFFileContentToTXT('CV.pdf')

In [10]:
pdfFileContent

'Luis Hernández ramosTuxtla Gutiérrez, Chis.luisramos170300@gmail.com9614486352Experiencia laboralIgualador automotriz y encargado de sucursalComercializadora de pinturas y complementos s.a de c.v  -  Tuxtla Gutiérrez, Chis. abril 2019 - enero 2022 Igualador de pintura automotriz en la línea ppg Esmaltes  acrílicos,vendedor  de  mostrador  con  buena  presentación,puntualidad  con responsabilidad,buena administración de una sucursal ,con todas las ganas de trabajar y demostrar un buen trabajo con la visión de escálalar Educación y formación Bachillerato trunco o en curso en Administración de recursos humanos Cetis 138  -  Tuxtla Gutiérrez, Chis. Habilidades, conocimientos e idiomas  •    manejo de computadora    •    microsoft office    •    manejo de excel    •    manejo de personal    •    microsoft office  '

### Separamos las palabras pegadas TXT

In [11]:
count = 1

for letter in range(len(pdfFileContent)): #funcion para limpiar y corregir errores de transformacion a txt
    
    count +=1
    if count == len(pdfFileContent):
        break
    
    l1 = pdfFileContent[letter]
    l2 = pdfFileContent[letter + 1]
    l3 = pdfFileContent[letter + 2]
    
    if l1.islower() == True and l2.isupper() == True and l3.islower() == True: #Errores de \n
        print("Errores por separar (letras): " + l1,l2,l3)
        pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)
        
    if l1.isdigit() == True and l2.isupper() == True and l3.islower() == True: #Numeros pegados a letras mayusculas 
        print("Errores por separar (numero): " + l1,l2,l3)
        pdfFileContent = pdfFileContent.replace(l1+l2+l3,l1+" "+l2+l3)

Errores por separar (letras): s T u
Errores por separar (numero): 2 E x
Errores por separar (letras): l I g
Errores por separar (letras): l C o


### Tokenize y removemos caracteres innecesarios (Tokens 1 y 2)

In [12]:
Tokens_1 = word_tokenize(pdfFileContent) 
#Hacemos token el CV de la persona y los requrimientos de las vacnates
Tokens_vacantes_1 = word_tokenize(requerimientos)

In [13]:
punctuation = re.compile(r'[-.?!,:;()•@[0-9]') #lista de caracteres que removeremos

Tokens_2 = []
for words in Tokens_1:
    word = punctuation.sub("",words)
    if len(word)>0:
        Tokens_2.append(word)
        
Tokens_vacantes_2 = []
for words in Tokens_vacantes_1:
    word = punctuation.sub("",words)
    if len(word)>0:
        Tokens_vacantes_2.append(word)

In [14]:
Tokens_2[10:15] #tokens con puras palabras sin caracteres

['automotriz', 'y', 'encargado', 'de', 'sucursal']

In [15]:
Tokens_vacantes_2[0:5]

['Igualación', 'de', 'colores', 'Captura', 'de']

### Lemmetize y remove stop words de el CV y los Requerimientos (Tokens 3 y 4)

In [16]:
stopw = stopwords.words("spanish") #Removemos las palabras de relleno que no aportan informacion
Tokens_3 = [word for word in Tokens_2 if word not in stopw] 
Tokens_vacantes_3 = [word for word in Tokens_vacantes_2 if word not in stopw] 

In [17]:
Tokens_3[10:15]

['automotriz', 'encargado', 'sucursal', 'Comercializadora', 'pinturas']

In [18]:
Tokens_vacantes_3[0:5]

['Igualación', 'colores', 'Captura', 'bitácoras', 'Trabajo']

In [19]:
sbst = SnowballStemmer('spanish')

Tokens_4 = []
for words in Tokens_3:
    word = sbst.stem(words)
    Tokens_4.append(word)
    #print(words+": "+ sbst.stem(words))
    
Tokens_vacantes_4 = []
for words in Tokens_vacantes_3:
    word = sbst.stem(words)
    Tokens_vacantes_4.append(word)

In [20]:
Tokens_4[10:15]

['automotriz', 'encarg', 'sucursal', 'comercializ', 'pintur']

In [21]:
Tokens_vacantes_4[0:5]

['igual', 'color', 'captur', 'bitacor', 'trabaj']

In [22]:
sentence_1 = " ".join(Tokens_4)
sentence_1

'luis hernandez ram tuxtl gutierrez chisluisr gmailcom experient laboral igual automotriz encarg sucursal comercializ pintur complement sa cv tuxtl gutierrez chis abril ener igual pintur automotriz line ppg esmalt acril vendedor mostrador buen present puntual respons buen administr sucursal tod gan trabaj demostr buen trabaj vision escalal educ formacion bachillerat trunc curs administr recurs human cetis tuxtl gutierrez chis habil conoc idiom manej comput microsoft offic manej excel manej personal microsoft offic'

In [23]:
sentence_2 = " ".join(Tokens_vacantes_4)
sentence_2

'igual color captur bitacor trabaj equip pintur automotriz'

### Buscar Requerimientos en CV | Bag Of Words

In [73]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

CountVec = CountVectorizer(ngram_range=(1,1)) # to use bigrams ngram_range=(2,2)

#transform
Count_data = CountVec.fit_transform([sentence_1,sentence_2])
 
#create dataframe
cv_dataframe = pd.DataFrame(Count_data.toarray(),columns = CountVec.get_feature_names_out())
cv_dataframe

,abril,acril,administr,automotriz,bachillerat,bitacor,buen,captur,cetis,chis,...,recurs,respons,sa,sucursal,tod,trabaj,trunc,tuxtl,vendedor,vision
0,1,1,2,2,1,0,3,0,1,2,...,1,1,1,2,1,2,1,3,1,1
1,0,0,0,1,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


In [82]:
count = 0
count2 = 0
matches = []

for i,j in zip(cv_dataframe.iloc[0],cv_dataframe.iloc[1]):
    if i >= 1 and j >=1:
        count += 1
        matches.append(cv_dataframe.columns.values[count2])
    count2 += 1

### Dar una Calificacion Basada en Los Requerimientos 

In [83]:
final_score = (count / len(Tokens_vacantes_4))*100

print("El perfil del vacante se alinea " + str(round(final_score)) + "% con el de los requerimeitnos" )

El perfil del vacante se alinea 50% con el de los requerimeitnos


In [84]:
matches

['automotriz', 'igual', 'pintur', 'trabaj']